In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Document Image Analysis/Assignment_1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

 1_600x900.jpg	 2_1000x1500.jpg   3_3700x5500.jpg  'Assignment 1 – Up-scaling Image.ipynb'


## UpScaling function

### First is nearest neighbor interpolation

In [3]:
import numpy as np

def nearest_neighbor_resize(image):
    scale = 2
    ori_height, ori_width = image.shape[:2]
    new_height, new_width = ori_height*scale, ori_width*scale
    resized_image = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)

    # # use loop is too slow, more than 1 hour
    # for i in range(ori_height):
    #     for j in range(ori_width):
    #         print(i, j)
    #         resized_image[2*i, 2*j], resized_image[2*i+1, 2*j], resized_image[2*i, 2*j+1], resized_image[2*i+1, 2*j+1] = image[i, j], image[i, j], image[i, j], image[i, j]

    # instead, use numpy
    resized_image[::scale, ::scale] = image
    resized_image[1::scale, ::scale] = image
    resized_image[::scale, 1::scale] = image
    resized_image[1::scale, 1::scale] = image

    return resized_image

### Second is bilinear interpolation

In [4]:
def bilinear_resize(image, up_rate):
    ori_height, ori_width = image.shape[:2]
    new_height, new_width = ori_height*up_rate, ori_width*up_rate
    resized_image = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)

    print(resized_image.shape)

    for i in range(new_height):
        for j in range(new_width):
            x = int(i * 1.0/up_rate)
            y = int(j * 1.0/up_rate)

            print(x, y)

            x_floor, y_floor = x, y
            x_ceil, y_ceil = min(x_floor + 1, ori_height - 1), min(y_floor + 1, ori_width - 1)

            tl = image[x_floor, y_floor]
            tr = image[x_floor, y_ceil]
            bl = image[x_ceil, y_floor]
            br = image[x_ceil, y_ceil]

            top = tl + (tr - tl) * (y - y_floor)
            bottom = bl + (br - bl) * (y - y_floor)
            pixel_value = top + (bottom - top) * (x - x_floor)

            resized_image[i, j] = np.clip(pixel_value, 0, 255)

    return resized_image

However, two for loop is two slow.

Try to implement with numpy

In [5]:
def bilinear_resize(image, scale=2):
    src_height, src_width = image.shape[:2]
    dst_height = src_height * scale
    dst_width = src_width * scale

    dst_x, dst_y = np.meshgrid(np.arange(dst_width), np.arange(dst_height))

    src_x = dst_x / scale
    src_y = dst_y / scale

    x0 = np.floor(src_x).astype(np.int64)
    x1 = x0 + 1
    y0 = np.floor(src_y).astype(np.int64)
    y1 = y0 + 1

    x0 = np.clip(x0, 0, src_width - 1)
    x1 = np.clip(x1, 0, src_width - 1)
    y0 = np.clip(y0, 0, src_height - 1)
    y1 = np.clip(y1, 0, src_height - 1)

    Ia = image[y0, x0]
    Ib = image[y1, x0]
    Ic = image[y0, x1]
    Id = image[y1, x1]

    # still some problem
    wa = (x1 - src_x) * (y1 - src_y) # bottom right - center
    wb = (x1 - src_x) * (src_y - y0) # right - center
    wc = (src_x - x0) * (y1 - src_y) # center -
    wd = (src_x - x0) * (src_y - y0)

    output = wa[..., np.newaxis] * Ia + wb[..., np.newaxis] * Ib + wc[..., np.newaxis] * Ic + wd[..., np.newaxis] * Id

    return output

## Load, up-scaling and save

In [6]:
from PIL import Image
import os

current_dir = '.'
methods = ['_Neighbor', '_Bilinear']
methods = ['_Neighbor']

for method in methods:
    for filename in os.listdir(current_dir):
        if filename.endswith('.jpg'):
            print(filename)
            image = Image.open(filename)
            image = np.array(image)
            if method == '_Neighbor':
              image_UpScaling = nearest_neighbor_resize(image)
            elif method == '_Bilinear':
              image_UpScaling = bilinear_resize(image)
            image_UpScaling = Image.fromarray(image_UpScaling)

            base_filename, file_extension = os.path.splitext(filename)
            new_filename = base_filename + method + file_extension

            image_UpScaling.save(new_filename)

3_3700x5500.jpg
2_1000x1500.jpg
1_600x900.jpg
